# Merging Pharmacy Files
**Author:** Marang Mutloatse

**Description:** This script merges the Pharmacy files from the 4 states.

**Version:** 0.0.1

**Status:** Completed

## Import Libraries

In [2]:
import pandas as pd
import datetime as dt
import warnings
import os
# get parent user path
user_dev_path = os.path.dirname(os.getcwd())
# get parent path for package
package_path = os.path.dirname(user_dev_path)
import seaborn as sns
import yaml
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

## Working Functions

In [3]:
def load_excel_sheet(path: str):
    if path is not None:
        try:
            xls = pd.ExcelFile(path)
            df = xls.parse(skiprows=0)
            return df
        except Exception as e:
            print(f"Exception on loading excel spreadsheet with error: {e}")

def load_files(file_name):
    if file_name.endswith('.csv'):
        df = pd.read_csv(file_name,low_memory=False)
    elif file_name.endswith('.xlsx'):
        df = load_excel_sheet(file_name)
    elif file_name.endswith('.psv'):
        df = pd.read_csv(file_name, sep = "|")
    else: 
        print('file format not supported')
    return df

## Loading Data

In [4]:
 try: 
    with open (user_dev_path + '/' + 'config_LTFU.yaml', 'r') as file:
        config = yaml.safe_load(file)
except Exception as e:
    print(f'Error reading the config file, {e}')

In [5]:
input_root = config['rise_files']['raw_data_path']
adamawa_file = config['rise_files']['raw_pharmacy_adamawa_file']
akwa_ibom_file = config['rise_files']['raw_pharmacy_akwa_ibom_file']
cross_river_file = config['rise_files']['raw_pharmacy_cross_river_file']
niger_file = config['rise_files']['raw_pharmacy_niger_file']
adamawa_input = input_root + adamawa_file
akwa_ibom_input = input_root + akwa_ibom_file
cross_river_input = input_root + cross_river_file
niger_input = input_root + niger_file
print(adamawa_input)
print(akwa_ibom_input)
print(cross_river_input)
print(niger_input)

/data/rise_data/PharmacyData_Adamawa.xlsx
/data/rise_data/PharmacyData_Akwa Ibom.xlsx
/data/rise_data/PharmacyData_Cross River.xlsx
/data/rise_data/PharmacyData_Niger.csv


In [6]:
# Load Adamawa
df_ada = load_files(adamawa_input)

In [7]:
# Load Akwa Ibom
df_akw = load_files(akwa_ibom_input)

In [9]:
df_akw.drop(columns=['Drug'],axis=1,inplace=True)

In [10]:
df_akw.rename(columns={'Regimen Type': 'Regimen Line'}, inplace=True)

In [11]:
df_akw['DMOC_TYPE'] = np.NaN
df_akw['BODY_WEIGHT'] = np.NaN

In [12]:
# Load Cross River
df_crs = load_files(cross_river_input)

In [14]:
# Load Niger
df_nig = load_files(niger_input)

In [15]:
df_nig.columns

Index(['name', 'name.1', 'name.2', 'regimentype', 'regimen', 'drug',
       'PHARMACY_ID', 'PATIENT_ID', 'FACILITY_ID', 'DATE_VISIT', 'DURATION',
       'MORNING', 'AFTERNOON', 'EVENING', 'ADR_SCREENED', 'ADR_IDS',
       'PRESCRIP_ERROR', 'ADHERENCE', 'NEXT_APPOINTMENT', 'DMOC_TYPE',
       'BODY_WEIGHT'],
      dtype='object')

In [16]:
df_nig.drop(columns=['drug'],axis=1,inplace=True)

In [17]:
df_nig.rename(columns={'name': 'State','name.1': 'L.G.A','name.2': 'Facility Name','regimentype': 'Regimen Line','regimen':'Regimen'}, inplace=True)

In [23]:
final_columns = [
    "State",
    "L.G.A",
    "Facility Name",
    "Regimen Line",
    "Regimen",
    "PHARMACY_ID",
    "PATIENT_ID",
    "FACILITY_ID",
    "DATE_VISIT",
    "DURATION",
    "MORNING",
    "AFTERNOON",
    "EVENING",
    "ADR_SCREENED",
    "ADR_IDS",
    "PRESCRIP_ERROR",
    "ADHERENCE",
    "NEXT_APPOINTMENT",
    "DMOC_TYPE",
    "BODY_WEIGHT",
]

In [25]:
df_nig_data = df_nig[final_columns]
df_akw_data = df_akw[final_columns]
df_crs_data = df_crs[final_columns]
df_ada_data = df_ada[final_columns]

In [27]:
df1 = pd.concat([df_nig_data, df_akw_data])
df2 = pd.concat([df_ada_data, df_crs_data])
df3 = pd.concat([df1, df2])

In [ ]:
df3.to_csv(input_root+'concact_pharmacy_data.csv',index=False,header=True)